In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory("../input/chest-xray-pneumonia/chest_xray/train",
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory("../input/chest-xray-pneumonia/chest_xray/test",
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'binary')


In [17]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory("../input/chest-xray-pneumonia/chest_xray/val",
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
callbacks = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
# autosave best Model
best_model_file = "../working/" + "_weightfile_finetuning.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)

In [53]:
model = Sequential([
    Conv2D(32,3, activation='relu', input_shape=(256,256,3)),
   
    MaxPooling2D(pool_size=(3, 3)),
    Conv2D(64,3, activation = 'relu'),
    Conv2D(128,3, activation = 'relu'),
    MaxPooling2D(pool_size=(3, 3)),
    Dropout(0.25),
    
    

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1,activation='sigmoid') #activation change
])

model.summary()

In [54]:
model.compile(optimizer='Adam', 
              loss='binary_crossentropy', 
              metrics =['accuracy'])

In [58]:
history = model.fit_generator(training_set,
                              epochs=10,
                              verbose=1,
                              validation_data=test_set,
                              callbacks = [best_model]
                              )

In [1]:
import matplotlib.pyplot as plt
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(14,7))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()